# Instalando Bibliotecas Adicionais

In [1]:
# !pip install altair-ally &> /dev/null
!pip install colorama &> /dev/null
# !pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip &> /dev/null
!pip install tabulate

# Importando Bibliotecas

In [2]:
import altair as alt
# import altair_ally as aly
import numpy as np
import pandas as pd
from colorama import Fore
from ipywidgets import widgets
# from pandas_profiling import ProfileReport
from scipy.stats import levene, mannwhitneyu, ranksums, ttest_ind, wilcoxon
from tabulate import tabulate

# Carregando os Dados

In [3]:
states = ['AC', 'AM', 'AP', 'MA', 'MT', 'PA', 'RO', 'RR', 'TO']
dfs_per_state = {}
dfs_per_state_till_2018 = {}

for state in states:
  df = pd.read_csv(
    f'https://raw.githubusercontent.com/george-gca/MO826_IA368_CF044/main/data/processed/maior_menor_desmat_{state}.csv')

  # finding name of cities with incomplete data
  columns = list(df.columns)
  columns.remove('Internações Ano Seguinte - Dengue')
  df_tmp = df[columns]
  cities_with_na = set(df_tmp[df_tmp.isna().any(axis=1)].Municipio.unique())

  df = df[~df.Municipio.isin(cities_with_na)]
  if len(df) > 0:
    dfs_per_state[state] = df
    dfs_per_state_till_2018[state] = df[df.Ano < 2019]
  else:
    print(f'Nenhum município completo no {state}')

states = list(dfs_per_state.keys())
dfs_per_state[states[0]]

Nenhum município completo no AP
Nenhum município completo no RR


,Lat,Long,Municipio,Estado,AreaKm2,Ano,Desmatado,Incremento,Floresta,NaoFloresta,Hidrografia,PercDesmatado,Internações - Dengue,Internações Ano Seguinte - Dengue,Grupo
0,-9.51622,-70.29089,Santa Rosa do Purus,AC,6350,2008,70.2,2.2,6206.8,0.1,34.9,0.011055,0.0,0.0,Menor
1,-10.71979,-70.01543,Assis Brasil,AC,5180,2008,247.6,3.5,4888.6,0.5,0.0,0.047799,2.0,2.0,Menor
2,-10.87940,-68.58788,Epitaciolândia,AC,1703,2008,811.9,4.4,881.4,0.0,0.1,0.476747,1.0,46.0,Menor
3,-9.39576,-70.06354,Manoel Urbano,AC,11118,2008,257.1,4.6,10796.8,0.0,49.5,0.023125,1.0,1.0,Maior
4,-7.45141,-73.41241,Mâncio Lima,AC,5762,2008,388.7,4.6,5156.1,45.1,0.0,0.067459,0.0,0.0,Menor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,-10.61178,-68.46285,Xapuri,AC,5511,2019,1498.1,62.3,4012.9,0.0,0.0,0.271838,26.0,NaN,Maior
236,-8.26015,-71.43408,Tarauacá,AC,21156,2019,1686.8,70.6,19454.4,1.2,13.6,0.079732,25.0,NaN,Maior
237,-9.99527,-68.15925,Rio Branco,AC,9113,2019,2881.9,82.6,6231.0,0.0,0.1,0.316241,23.0,NaN,Maior
238,-9.77147,-69.33204,Sena Madureira,AC,24599,2019,2085.5,83.1,22510.5,2.4,0.6,0.084780,26.0,NaN,Maior


# Verificando a Distribuição dos Dados de Internações por Dengue

In [4]:
def display_in_matrix(plots):
  matrix_result = [plots[0]]
  for i, p in enumerate(plots[1:]):
    if i % 2 == 0:
      matrix_result[i//2] = matrix_result[i//2] | p
    else:
      matrix_result.append(p)
    
  result = matrix_result[0]
  for r in matrix_result[1:]:
    result = result & r

  return result

In [5]:
plots = []
for state in states:
  df = dfs_per_state[state]

  for group in ['Menor', 'Maior']:
    plots.append(alt.Chart(df[df.Grupo == group]).mark_bar().encode(
        alt.X('Internações - Dengue:O', bin=True),
        y='count()',
    ).properties(
        title=f'{group}es ({state})'
    ))

display_in_matrix(plots)

alt.VConcatChart(...)

Os dados das amostras foram submetidos à visualização a partir de histogramas, sendo possível observar que suas frequências apresentam uma distribuição assimétrica. Os histogramas apontaram para dados concentrados à esquerda, ou seja, os menores valores da amostra possuem as frequencias mais altas. Na prática isso indica que, em nossa amostra, foram contabilizados menos casos de internações por dengue mais frequentemente.

# Teste de Hipóteses

Para realização do teste de hipóteses bicaudal, alguns fatores como, por exemplo, tamanho amostral, distribuição de variâncias e distribuição de frequências foram considerados.


Apesar da nossa amostra não estar de acordo com um dos pressupostos do teste T, que é a distribuição normal dos dados, sabe-se que esse teste possui robustez suficiente para lidar com dados não simétricos quando outros fatores são observados como, por exemplo, N amostral grande, homogeneidade do N entre os grupos, homogeneidade entre variâncias. Com relação a homogeneidade de variâncias, o teste de Levene foi realizado e seu resultado considerado durante a execução do teste T.

Os dois grupos analisados foram definidos a partir dos maiores e menores índices de desmatamento na Amazônia, ou seja, as variáveis independentes do estudo. Em contrapartida, os números de internações por dengue foram selecionados como variáveis dependentes (que podem sofrer efeito dos grupos).

As hipóteses levantadas foram:


H0 = Não existe diferença entre as médias dos casos de de internação por dengue na Amazônia entre os grupos de municípios de menor e maior desmatamento.

(A média dos casos de internação por dengue na Amazônia não é influenciada pelo desmatamento).

H1 = Existe diferença entre as médias dos casos de de internação por dengue na Amazônia entre os grupos de municípios de menor e maior desmatamento.

(A média dos casos de internação por dengue na Amazônia é influenciada pelo desmatamento).

In [6]:
def hypothesis_test(df, column, id):
  df_maior = df[df.Grupo == 'Maior'][column]
  df_menor = df[df.Grupo == 'Menor'][column]
  _, p_levene = levene(df_maior, df_menor)

  stat, p_ttest = ttest_ind(
    df_maior,
    df_menor,
    axis=None,
    equal_var=p_levene < 0.05)
  
  if p_ttest < 0.05:
    result = f'{Fore.GREEN}Sim{Fore.RESET}'
  else:
    result = f'{Fore.RED}Não{Fore.RESET}'
  
  return {id[0]: id[1],
          'Média (Menor)': df_menor.mean(),
          'Média (Maior)': df_maior.mean(),
          't-valor': stat,
          'p-valor': p_ttest,
          'Existe diferença entre a média dos grupos': result}

In [7]:
rows = []
for state in states:
  rows.append(hypothesis_test(dfs_per_state[state], 'Internações - Dengue', ('Estado', state)))

print(tabulate(rows, headers='keys', tablefmt='github'))

| Estado   |   Média (Menor) |   Média (Maior) |   t-valor |     p-valor | Existe diferença entre a média dos grupos   |
|----------|-----------------|-----------------|-----------|-------------|---------------------------------------------|
| AC       |       10.8487   |        21.8595  |   1.51252 | 0.131729    | Não                                         |
| AM       |        0.333333 |         4.48333 |   3.09707 | 0.00220179  | Sim                                         |
| MA       |       11.6417   |        17.7417  |   2.0323  | 0.0434698   | Sim                                         |
| MT       |        3.99074  |        16.6083  |   4.14621 | 4.78304e-05 | Sim                                         |
| PA       |       33.45     |        62.8417  |   2.06063 | 0.0404487   | Sim                                         |
| RO       |       51.9667   |        22.7833  |  -3.02313 | 0.00277554  | Sim                                         |
| TO       |       10.5682   |  

A partir do teste de hipótese, quando avaliamos os índices de internação por dengue e desmatamento no mesmo ano, foi possível observar diferenças entre médias estatisticamente significativas em 5/7 estados. Dentre esses, em 4 estados (AM, MA, MT, PA) os municípios com maiores índices de desmatamento também tiveram maiores índices de internações por dengue. Já no estado de RO foi observado que os municípios com menores índices de desmatamento tinham maiores índices de internações. Nos estados do AC e TO, não foram observadas relações estatisitcamente significativas entre as variáveis estudadas.

In [8]:
rows = []
for state in states:
  rows.append(hypothesis_test(dfs_per_state_till_2018[state],
                              'Internações Ano Seguinte - Dengue',
                              ('Estado', state)))
  
print(tabulate(rows, headers='keys', tablefmt='github'))

| Estado   |   Média (Menor) |   Média (Maior) |   t-valor |     p-valor | Existe diferença entre a média dos grupos   |
|----------|-----------------|-----------------|-----------|-------------|---------------------------------------------|
| AC       |        11.7523  |        21.8739  |   1.29663 | 0.19613     | Não                                         |
| AM       |         0.20202 |         4.74545 |   3.13384 | 0.00197565  | Sim                                         |
| MA       |        11.9091  |        17.5909  |   1.76997 | 0.078435    | Não                                         |
| MT       |         4.16162 |        16.7091  |   3.88925 | 0.000135471 | Sim                                         |
| PA       |        30.1636  |        64.1273  |   2.32834 | 0.0208144   | Sim                                         |
| RO       |        54.3     |        23.5364  |  -2.95714 | 0.0034471   | Sim                                         |
| TO       |        11.1901  |  

A partir do teste de hipótese, quando avaliamos os índices de internação por dengue no ano seguinte ao desmatamento, foi possível observar diferenças entre médias estatisticamente significativas em 4/7 estados. Dentre esses, em 3 estados (AM, MT, PA) os municípios com maiores índices de desmatamento também tiveram maiores índices de internações por dengue. Já no estado de RO foi observado que os municípios com menores índices de desmatamento tinham maiores índices de internações. Nos estados do AC e TO, não foram observadas relações estatisitcamente significativas entre as variáveis estudadas.

Agrupando agora os dados por ano, independente do estado

In [9]:
dfs_per_year = {}
years = list(range(2008, 2020))

for year in years:
  dfs_tmp = []

  for state in states:
    df = dfs_per_state[state]
    df = df[df.Ano == year]
    dfs_tmp.append(df)
  
  dfs_per_year[year] = pd.concat(dfs_tmp)

dfs_per_year[years[0]]

,Lat,Long,Municipio,Estado,AreaKm2,Ano,Desmatado,Incremento,Floresta,NaoFloresta,Hidrografia,PercDesmatado,Internações - Dengue,Internações Ano Seguinte - Dengue,Grupo
0,-9.51622,-70.29089,Santa Rosa do Purus,AC,6350,2008,70.2,2.2,6206.8,0.1,34.9,0.011055,0.0,0.0,Menor
1,-10.71979,-70.01543,Assis Brasil,AC,5180,2008,247.6,3.5,4888.6,0.5,0.0,0.047799,2.0,2.0,Menor
2,-10.87940,-68.58788,Epitaciolândia,AC,1703,2008,811.9,4.4,881.4,0.0,0.1,0.476747,1.0,46.0,Menor
3,-9.39576,-70.06354,Manoel Urbano,AC,11118,2008,257.1,4.6,10796.8,0.0,49.5,0.023125,1.0,1.0,Maior
4,-7.45141,-73.41241,Mâncio Lima,AC,5762,2008,388.7,4.6,5156.1,45.1,0.0,0.067459,0.0,0.0,Menor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,-5.78728,-47.60538,Itaguatins,TO,745,2008,356.2,1.6,67.9,310.0,10.9,0.478121,4.0,21.0,Maior
16,-12.11997,-50.36955,Formoso do Araguaia,TO,13453,2008,183.8,2.2,478.0,12257.6,167.8,0.013662,6.0,4.0,Maior
17,-9.37187,-49.84810,Caseara,TO,1697,2008,264.0,2.5,319.8,1079.1,34.0,0.155569,0.0,1.0,Maior
18,-8.76489,-48.40596,Guaraí,TO,2278,2008,1017.2,4.5,180.0,1077.7,3.0,0.446532,5.0,2.0,Maior


In [10]:
rows = []
for year in years:
  rows.append(hypothesis_test(dfs_per_year[year], 'Internações - Dengue', ('Ano', year)))

print(tabulate(rows, headers='keys', tablefmt='github'))

|   Ano |   Média (Menor) |   Média (Maior) |   t-valor |   p-valor | Existe diferença entre a média dos grupos   |
|-------|-----------------|-----------------|-----------|-----------|---------------------------------------------|
|  2008 |        16.2899  |        17.7246  |  0.148341 |  0.882372 | Não                                         |
|  2009 |        25.2286  |        30.1471  |  0.407    |  0.684657 | Não                                         |
|  2010 |        34       |        52.1     |  1.03647  |  0.301872 | Não                                         |
|  2011 |        26.8261  |        37.7536  |  0.740015 |  0.460644 | Não                                         |
|  2012 |        22.4058  |        26.058   |  0.327539 |  0.743764 | Não                                         |
|  2013 |        27.2206  |        25.2     | -0.172716 |  0.863161 | Não                                         |
|  2014 |        20.7714  |        16.1912  | -0.455021 |  0.65008  | Nã

In [11]:
rows = []
for year in years[:-1]:
  rows.append(hypothesis_test(dfs_per_year[year], 'Internações Ano Seguinte - Dengue', ('Ano', year)))

print(tabulate(rows, headers='keys', tablefmt='github'))

|   Ano |   Média (Menor) |   Média (Maior) |   t-valor |   p-valor | Existe diferença entre a média dos grupos   |
|-------|-----------------|-----------------|-----------|-----------|---------------------------------------------|
|  2008 |        25.3333  |        30.2754  |  0.408116 |  0.683848 | Não                                         |
|  2009 |        33.1286  |        50.25    |  0.986951 |  0.325431 | Não                                         |
|  2010 |        27.2059  |        38.3     |  0.748305 |  0.455675 | Não                                         |
|  2011 |        22.4058  |        26.058   |  0.327539 |  0.743764 | Não                                         |
|  2012 |        26.8261  |        25.4638  | -0.116957 |  0.907083 | Não                                         |
|  2013 |        21.3824  |        15.7429  | -0.550268 |  0.583432 | Não                                         |
|  2014 |        15.0714  |        16.6912  |  0.249946 |  0.803032 | Nã

Ao avaliarmos os dados por ano sem levarmos em consideração os estados, não foi possível observar resultados estatisticamente significativos entre os municípios com maiores e menores índices de desmatamento e internações por dengue. Esses resultados se mantiveram quando avaliamos as internações por dengue no ano seguinte ao desmatamento.